<a href="https://colab.research.google.com/github/aliikhwan99/Different_stage_of_AD_detection/blob/main/Different_stage_of_AD_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import os
import shutil

nooraliikhwan_stage_of_alzheimer_path = kagglehub.dataset_download('nooraliikhwan/stage-of-alzheimer')

print('Data source import complete.')


# Step-by-Step Process

1. Convert DICOM to NIfTI (if necessary)
2. Preprocess the rs-fMRI data: Load the NIfTI file and process the fMRI data to extract time series or compute functional connectivity.
3. Visualize brain surfaces: Project the functional data to cortical surfaces.
4. Create Functional Connectivity Matrices: Compute correlation matrices to represent functional connectivity between brain regions.

# Compulsory Libraries

1. nibabel: For loading and handling DICOM/NIfTI files.
2. nilearn: For preprocessing, visualizing, and computing functional connectivity.
3. numpy: For matrix operations.
4. matplotlib: For visualizations.

# Averaging across time (optional)

In [ ]:


!dcm2niix -o /kaggle/working/ -f fmri_image /kaggle/input/stage-of-alzheimer/EMCI/ADNI/002_S_2010/Resting_State_fMRI/2010-06-24_14_21_28.0/I180734


In [ ]:
from nilearn import image, datasets, surface, plotting

# Load your newly converted NIfTI image
fmri_img = image.load_img('/kaggle/working/fmri_image.nii')

# Select the first volume (if it's a 4D image, e.g., rs-fMRI data)
fmri_img_vol = image.index_img(fmri_img, 0)

# Load a standard brain surface for visualization
fsaverage = datasets.fetch_surf_fsaverage()

# Project the selected volume to the cortical surface
texture_left = surface.vol_to_surf(fmri_img_vol, fsaverage.pial_left)
texture_right = surface.vol_to_surf(fmri_img_vol, fsaverage.pial_right)

# Plot the projected data on the brain surface
plotting.plot_surf_stat_map(fsaverage.infl_left, texture_left, hemi='left', title='Left Hemisphere')
plotting.plot_surf_stat_map(fsaverage.infl_right, texture_right, hemi='right', title='Right Hemisphere')

plotting.show()


# Visualize DICOM file (rs-fMRI)

In [ ]:
from nilearn import image

# Load your fMRI image
fmri_img = image.index_img(fmri_img, 0)  # Select the first volume (or another index)

# Project the selected volume to the cortical surface
texture_left = surface.vol_to_surf(fmri_img, fsaverage.pial_left)
texture_right = surface.vol_to_surf(fmri_img, fsaverage.pial_right)

# Plot the projected data on the brain surface
plotting.plot_surf_stat_map(fsaverage.infl_left, texture_left, hemi='left', title='Left Hemisphere')
plotting.plot_surf_stat_map(fsaverage.infl_right, texture_right, hemi='right', title='Right Hemisphere')

plotting.show()


# Convert DICOM files to NIfTI using the dcm2niix tool

In [ ]:
!apt-get install -y dcm2niix


In [ ]:
import subprocess

dicom_directory = '/kaggle/input/stage-of-alzheimer/NC/ADNI/002_S_0295/Resting_State_fMRI/2011-06-02_07_56_36.0/I238623'
output_directory = '/kaggle/working'  # Specify where you want to save the NIfTI file

# Run dcm2niix and capture output and error messages
result = subprocess.run(['dcm2niix', '-z', 'y', '-f', '%p_%s', '-o', output_directory, dicom_directory],
                        capture_output=True, text=True)

# Print the output and error messages
print("Output:\n", result.stdout)
print("Error:\n", result.stderr)


In [ ]:
import os

print("Converted files:")
print(os.listdir(output_directory))


In [ ]:
import os
import pydicom
import matplotlib.pyplot as plt

# Define your DICOM directory
dicom_directory = '/kaggle/input/stage-of-alzheimer/MCI/ADNI/002_S_1155/Resting_State_fMRI/2012-12-20_12_16_12.0/I352947'

# List all files in the directory
files = os.listdir(dicom_directory)

# Filter to only get DICOM files (optional if there are non-DICOM files)
dicom_files = [f for f in files if f.endswith('.dcm')]

# Check if any DICOM files exist
if len(dicom_files) == 0:
    print("No DICOM files found in the directory.")
else:
    # Load the first DICOM file
    dicom_file = os.path.join(dicom_directory, dicom_files[0])
    ds = pydicom.dcmread(dicom_file)

    # Print metadata
    print(ds)

    # Extract and plot the image data (if available)
    plt.imshow(ds.pixel_array, cmap=plt.cm.gray)
    plt.show()


# Normal

In [ ]:
import os
import pydicom
import matplotlib.pyplot as plt

# Define your DICOM directory
dicom_directory = '/kaggle/input/stage-of-alzheimer/NC/ADNI/002_S_0295/Resting_State_fMRI/2011-06-02_07_56_36.0/I238623'

# List all files in the directory
files = os.listdir(dicom_directory)

# Filter to only get DICOM files (optional if there are non-DICOM files)
dicom_files = [f for f in files if f.endswith('.dcm')]

# Check if any DICOM files exist
if len(dicom_files) == 0:
    print("No DICOM files found in the directory.")
else:
    # Load the first DICOM file
    dicom_file = os.path.join(dicom_directory, dicom_files[1])
    ds = pydicom.dcmread(dicom_file)

    # Print metadata
    print(ds)

    # Extract and plot the image data (if available)
    plt.imshow(ds.pixel_array, cmap=plt.cm.gray)
    plt.show()


# Visualizing the Connectivity Matrix

# Normal

In [ ]:
import os
import numpy as np
import pydicom  # Make sure pydicom is installed in your environment
import matplotlib.pyplot as plt
import seaborn as sns
from nilearn import datasets
from scipy.ndimage import gaussian_filter  # Import Gaussian filter

# Path to the directory containing the DICOM files
dicom_directory = '/kaggle/input/stage-of-alzheimer/NC/ADNI/002_S_0295/Resting_State_fMRI/2011-06-02_07_56_36.0/I238623'

# Check if the directory exists
if not os.path.exists(dicom_directory):
    print(f"Directory not found: {dicom_directory}")
else:
    print(f"Directory exists: {dicom_directory}")

    # Load the DICOM files
    def load_dicom_series(directory):
        dicom_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.dcm')]
        dicom_files.sort()  # Sort the files to maintain order
        images = []
        for dicom_file in dicom_files:
            ds = pydicom.dcmread(dicom_file)
            images.append(ds.pixel_array)  # Append pixel data
        return np.array(images)

    # Load your fMRI data
    fmri_data = load_dicom_series(dicom_directory)
    print(f"Shape of fMRI data: {fmri_data.shape}")  # (time_points, height, width)

# Function to extract time series from the 4D fMRI data
def extract_time_series(fmri_data):
    # Get the shape of the fMRI data
    time_points, height, width = fmri_data.shape[0], fmri_data.shape[1], fmri_data.shape[2]

    # Reshape to a 2D array: (voxels, time_points)
    reshaped_data = fmri_data.reshape(-1, time_points)
    return reshaped_data

# Extract time series from the fMRI data
time_series = extract_time_series(fmri_data)
print(f"Shape of time series data: {time_series.shape}")  # (voxels, time_points)

# Function to compute the functional connectivity matrix
def compute_functional_connectivity(time_series):
    # Calculate the correlation matrix
    connectivity_matrix = np.corrcoef(time_series)
    return connectivity_matrix

# Compute the functional connectivity matrix
connectivity_matrix = compute_functional_connectivity(time_series)
print(f"Shape of connectivity matrix: {connectivity_matrix.shape}")  # (voxels, voxels)

# Apply Gaussian filter to the functional connectivity matrix
smoothed_connectivity_matrix = gaussian_filter(connectivity_matrix, sigma=2)  # You can adjust sigma value

# Load the AAL brain atlas and its labels
atlas = datasets.fetch_atlas_aal()
atlas_labels = atlas['labels']  # Get brain region labels

# Truncate the connectivity matrix if it's larger than the number of available atlas labels
num_labels = len(atlas_labels)
if smoothed_connectivity_matrix.shape[0] > num_labels:
    truncated_connectivity_matrix = smoothed_connectivity_matrix[:num_labels, :num_labels]
else:
    truncated_connectivity_matrix = smoothed_connectivity_matrix  # Use the original if no truncation needed

# Function to plot a clearer connectivity matrix with enhancements
def plot_enhanced_connectivity_matrix(connectivity_matrix, labels):
    plt.figure(figsize=(14, 12), dpi=100)  # Increase size and DPI
    sns.set(font_scale=1)  # Increase font scale for better readability
    sns.heatmap(connectivity_matrix, cmap='coolwarm', square=True,
                xticklabels=labels[:connectivity_matrix.shape[0]],
                yticklabels=labels[:connectivity_matrix.shape[0]],
                cbar_kws={"shrink": .4}, linewidths=.2)  # Add lines between cells for clarity
    plt.title('Functional Connectivity Matrix (Enhanced)', fontsize=5)
    plt.xlabel('Brain Regions', fontsize=4)
    plt.ylabel('Brain Regions', fontsize=4)
    plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
    plt.tight_layout()  # Adjust layout to fit everything well
    plt.show()

# Plot the enhanced connectivity matrix with the AAL labels
plot_enhanced_connectivity_matrix(truncated_connectivity_matrix, atlas_labels)


In [ ]:
import os
import numpy as np
import pydicom  # Make sure pydicom is installed in your environment
import matplotlib.pyplot as plt
import seaborn as sns
from nilearn import datasets
from scipy.ndimage import gaussian_filter  # Import Gaussian filter
from scipy.ndimage import median_filter

# Path to the directory containing the DICOM files
dicom_directory = '/kaggle/input/stage-of-alzheimer/NC/ADNI/002_S_0295/Resting_State_fMRI/2011-06-02_07_56_36.0/I238623'

# Check if the directory exists
if not os.path.exists(dicom_directory):
    print(f"Directory not found: {dicom_directory}")
else:
    print(f"Directory exists: {dicom_directory}")

    # Load the DICOM files
    def load_dicom_series(directory):
        dicom_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.dcm')]
        dicom_files.sort()  # Sort the files to maintain order
        images = []
        for dicom_file in dicom_files:
            ds = pydicom.dcmread(dicom_file)
            images.append(ds.pixel_array)  # Append pixel data
        return np.array(images)

    # Load your fMRI data
    fmri_data = load_dicom_series(dicom_directory)
    print(f"Shape of fMRI data: {fmri_data.shape}")  # (time_points, height, width)

# Function to extract time series from the 4D fMRI data
def extract_time_series(fmri_data):
    # Get the shape of the fMRI data
    time_points, height, width = fmri_data.shape[0], fmri_data.shape[1], fmri_data.shape[2]

    # Reshape to a 2D array: (voxels, time_points)
    reshaped_data = fmri_data.reshape(-1, time_points)
    return reshaped_data

# Extract time series from the fMRI data
time_series = extract_time_series(fmri_data)
print(f"Shape of time series data: {time_series.shape}")  # (voxels, time_points)

# Function to compute the functional connectivity matrix
def compute_functional_connectivity(time_series):
    # Calculate the correlation matrix
    connectivity_matrix = np.corrcoef(time_series)
    return connectivity_matrix

# Apply median filter to reduce noise
filtered_matrix = median_filter(connectivity_matrix, size=3)  # Adjust size for the window size

# Compute the functional connectivity matrix
connectivity_matrix = compute_functional_connectivity(time_series)
print(f"Shape of connectivity matrix: {connectivity_matrix.shape}")  # (voxels, voxels)

# Apply Gaussian filter to the functional connectivity matrix
smoothed_connectivity_matrix = gaussian_filter(connectivity_matrix, sigma=3)  # You can adjust sigma value

# Load the AAL brain atlas and its labels
atlas = datasets.fetch_atlas_aal()
atlas_labels = atlas['labels']  # Get brain region labels

# Truncate the connectivity matrix if it's larger than the number of available atlas labels
num_labels = len(atlas_labels)
if smoothed_connectivity_matrix.shape[0] > num_labels:
    truncated_connectivity_matrix = smoothed_connectivity_matrix[:num_labels, :num_labels]
else:
    truncated_connectivity_matrix = smoothed_connectivity_matrix  # Use the original if no truncation needed

# Function to plot a clearer connectivity matrix with enhancements
def plot_enhanced_connectivity_matrix(connectivity_matrix, labels):
    plt.figure(figsize=(14, 12), dpi=100)  # Increase size and DPI
    sns.set(font_scale=1)  # Increase font scale for better readability
    sns.heatmap(connectivity_matrix, cmap='coolwarm', square=True,
                xticklabels=labels[:connectivity_matrix.shape[0]],
                yticklabels=labels[:connectivity_matrix.shape[0]],
                cbar_kws={"shrink": .4}, linewidths=.2)  # Add lines between cells for clarity
    plt.title('Functional Connectivity Matrix (Enhanced)', fontsize=5)
    plt.xlabel('Brain Regions', fontsize=4)
    plt.ylabel('Brain Regions', fontsize=4)
    plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
    plt.tight_layout()  # Adjust layout to fit everything well
    plt.show()

# Plot the enhanced connectivity matrix with the AAL labels
plot_enhanced_connectivity_matrix(truncated_connectivity_matrix, atlas_labels)


# Actual brain activity position

In [ ]:
pip install nilearn matplotlib


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nilearn import plotting, datasets, surface
from nilearn.input_data import NiftiLabelsMasker

# Mock example of your connectivity matrix; replace with your actual matrix
connectivity_matrix = np.random.rand(90, 90)  # Example connectivity matrix (90 regions)

# Load the AAL atlas
atlas = datasets.fetch_atlas_aal()
atlas_filename = atlas['maps']  # AAL atlas NIfTI file
atlas_labels = atlas['labels']  # Get brain region labels

# Load a standard brain surface for visualization
fsaverage = datasets.fetch_surf_fsaverage()

# Use NiftiLabelsMasker to extract time series data based on atlas regions
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True)

# Prepare a 1D array to represent the functional connectivity on the brain surface
left_connectivity = np.zeros(45)  # For left hemisphere
right_connectivity = np.zeros(45)  # For right hemisphere

# Map connectivity values to left and right hemispheres
for i in range(len(atlas_labels)):
    if i < 45:  # Left hemisphere
        left_connectivity[i] = connectivity_matrix[i, :].mean()  # Example aggregation
    elif i < 90:  # Right hemisphere
        right_index = i - 45  # Adjust index for right hemisphere
        right_connectivity[right_index] = connectivity_matrix[i, :].mean()  # Example aggregation

# Project the AAL atlas regions onto the brain surface
texture_left = surface.vol_to_surf(atlas_filename, fsaverage.pial_left)
texture_right = surface.vol_to_surf(atlas_filename, fsaverage.pial_right)

# Plot the left hemisphere
plotting.plot_surf_stat_map(fsaverage['infl_left'],
                            stat_map=texture_left,
                            hemi='left',
                            view='lateral',
                            title='Left Hemisphere',
                            colorbar=True)

# Plot the right hemisphere
plotting.plot_surf_stat_map(fsaverage['infl_right'],
                            stat_map=texture_right,
                            hemi='right',
                            view='lateral',
                            title='Right Hemisphere',
                            colorbar=True)

plt.show()
